# Problem setting

In this tutorial, we demonstrate how graph neural networks can be used for recommendation. Here we focus on item-based recommendation model. This method in this tutorial recommends items that are similar to the ones purchased by the user. We demonstrate the recommendation model on the MovieLens dataset.

# Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works with Pytorch.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL, we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [2]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Load the rest of necessary libraries.

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse as spsp

# Dataset

We use the MoiveLens dataset for demonstration because it is commonly used for recommendation models. In this dataset, there are two types of nodes: users and movies. The movie nodes have three attributes: year, title and genre. There are ratings between user nodes and movie nodes. Each rating has a timestamp. In our recommendation model, we don't consider ratings and timestamps.

**Note**: It is not necessarily the best dataset to demonstrate the power of GNN for recommendation. We have prepared the dataset to simplify the demonstration.

To run the data preprocessing script, a user needs to download the English dictionary of the stanfordnlp package first. However, the following command only needs to run once.

In [ ]:
# Please uncomment the two commands when the tutorial is run for the first time.
#import stanfordnlp
#stanfordnlp.download('en')

Load the MovieLens dataset.

In [ ]:
from movielens import MovieLens
data = MovieLens('.')

Calculate some statistics of the dataset.

In [ ]:
ratings = data.ratings
user_id = np.array(ratings['user_idx'])
movie_id = np.array(ratings['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
num_users, num_movies = user_movie_spm.shape
print('#user-movie iterations:', len(movie_id))
print('#users:', num_users)
print('#movies:', num_movies)

We split the dataset into a training set and a testing set

In [ ]:
ratings_train = ratings[~(ratings['valid_mask'] | ratings['test_mask'])]
user_latest_item_indices = (
        ratings_train.groupby('user_id')['timestamp'].transform(pd.Series.max) ==
        ratings_train['timestamp'])
user_latest_item = ratings_train[user_latest_item_indices]
user_latest_item = dict(
        zip(user_latest_item['user_idx'].values, user_latest_item['movie_idx'].values))

Construct the training, validation and testing dataset

In [ ]:
# The training dataset
user_id = np.array(ratings_train['user_idx'])
movie_id = np.array(ratings_train['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
assert num_users == user_movie_spm.shape[0]
assert num_movies == user_movie_spm.shape[1]
train_size = len(user_id)
print('#training size:', train_size)

# The validation and testing dataset
users_valid = ratings[ratings['valid_mask']]['user_idx'].values
movies_valid = ratings[ratings['valid_mask']]['movie_idx'].values
users_test = ratings[ratings['test_mask']]['user_idx'].values
movies_test = ratings[ratings['test_mask']]['movie_idx'].values
valid_size = len(users_valid)
test_size = len(users_test)

## Load MovieLens from pickle

In [118]:
import pickle
user_movie_spm = pickle.load(open('movielens/movielens_orig_train.pkl', 'rb'))

num_users = user_movie_spm.shape[0]
num_movies = user_movie_spm.shape[1]

In [119]:
features = pickle.load(open('movielens/movielens_features.pkl', 'rb'))
valid_set, test_set = pickle.load(open('movielens/movielens_eval.pkl', 'rb'))
neg_valid, neg_test = pickle.load(open('movielens/movielens_neg.pkl', 'rb'))

users_valid = np.arange(num_users)
movies_valid = valid_set
users_test = np.arange(num_users)
movies_test = test_set

movie_popularity = user_movie_spm.transpose().dot(np.ones(shape=(num_users)))
# We need to rescale the values
movie_popularity = torch.tensor(movie_popularity / np.max(movie_popularity), dtype=torch.float32).unsqueeze(1)

u, s, vt = spsp.linalg.svds(user_movie_spm)
v = torch.tensor(vt.transpose(), dtype=torch.float32)
v = v * torch.tensor(np.sqrt(s).transpose(), dtype=torch.float32)

features = torch.cat([features, movie_popularity, v], 1)
#one_hot = torch.tensor(np.diag(np.ones(shape=(num_movies))), dtype=torch.float32)
#features = torch.cat([features, one_hot], 1)
in_feats = features.shape[1]
print('#feats:', in_feats)

#feats: 4132


In [120]:
spm = user_movie_spm.tocsr()
user_latest_item = np.zeros((num_users), dtype=np.int64)
for i in range(num_users):
    row_start = spm.indptr[i]
    row_end = spm.indptr[i+1]
    user_latest_item[i] = np.random.choice(spm.indices[row_start:row_end])

In [121]:
print(len(np.unique(user_latest_item)))
print(len(user_latest_item))

1741
6040


Down sample user-movie pairs.

In [122]:
downsample_factor = 1e-6

def downsample(user_movie_spm):
    user_id = user_movie_spm.row
    movie_id = user_movie_spm.col
    spm_t = user_movie_spm.transpose()
    movie_deg = spm_t.dot(np.ones((num_users,)))
    movie_ratio = movie_deg / np.sum(movie_deg)
    # 1e-6 is a hyperparameter for this dataset.
    movie_sample_prob = 1 - np.maximum(1 - np.sqrt(downsample_factor / movie_ratio), 0)
    sample_prob = movie_sample_prob[movie_id]
    sample = np.random.uniform(size=(len(movie_id),))
    user_id = user_id[sample_prob > sample]
    movie_id = movie_id[sample_prob > sample]
    print('#samples:', len(user_id))
    user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
    print(user_movie_spm.shape)
    return user_movie_spm

user_movie_spm = downsample(user_movie_spm)

#samples: 48294
(6040, 3702)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


# The recommendation model

At large, the model first learns item embeddings from the user-item interaction dataset and use the item embeddings to recommend users similar items they have purchased. To learn item embeddings, we first need to construct an item similarity graph and train GNN on the item graph.

There are many ways of constructing the item similarity graph. Here we use the [SLIM model](https://dl.acm.org/citation.cfm?id=2118303) to learn item similarity and use the learned result to construct the item graph. The resulting graph will have an edge between two items if they are similar and the edge has a weight that represents the similarity score.

After the item similarity graph is constructed, we run a GNN model on it and use the vertex connectivity as the training signal to train the GNN model. The GNN training procedure is very similar to the link prediction task in [the previous section](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

## Construct the movie graph with SLIM
SLIM is an item-based recommendation model. When training SLIM on a user-item dataset, it learns an item similarity graph. This similarity graph is the item graph we construct for the GNN model.

Please follow the instruction on the [SLIM github repo](https://github.com/KarypisLab/SLIM) to install SLIM.

The SLIM only needs to run once and can be saved on the disk for future experiments.

In [142]:
from SLIM import SLIM, SLIMatrix
from slim_load import read_csr

def gen_slim_graph(user_movie_spm):
    model = SLIM()
    params = {'algo': 'cd', 'nthreads': 32, 'l1r': 1, 'l2r': 1.0}
    trainmat = SLIMatrix(user_movie_spm.tocsr())
    model.train(params, trainmat)
    model.save_model(modelfname='slim_model.csr', mapfname='slim_map.csr')

    # Load the SLIM similarity matrix into DGL. We store the vertex similarity as edge data on DGL.
    movie_spm = read_csr('slim_model.csr')
    print('SLIM graph has #edges:', movie_spm.nnz)
    print('most similar:', np.max(movie_spm.data))
    print('most unsimilar:', np.min(movie_spm.data))

    # Generate DGLGraph
    g = dgl.DGLGraph(movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
    return g

## Construct the co-occurance graph

In [124]:
def gen_cooccur_graph(user_movie_spm):
    #user_movie_spm = downsample(user_movie_spm)
    movie_spm = np.dot(user_movie_spm.transpose(), user_movie_spm)
    #dense_movie = np.sort(movie_spm.todense())
    #topk_movie = dense_movie[:,-50]
    #movie_spm1 = movie_spm >= topk_movie
    
    g = dgl.DGLGraph(movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
    return g

## Construct the cosine-similarity graph

In [125]:
from sklearn.metrics.pairwise import cosine_similarity
topk = 50
def gen_cosine_graph(user_movie_spm):
    movie_spm = cosine_similarity(user_movie_spm.transpose(),dense_output=False)
    topk_movie_spm = movie_spm.multiply(movie_spm > 0.3)
    #dense_movie = np.sort(movie_spm.todense())
    #topk_movie = dense_movie[:,-topk]
    #topk_movie_spm = movie_spm > topk_movie
    #topk_movie_spm = spsp.coo_matrix(topk_movie_spm)
    #print(type(topk_movie_spm))
    #topk_movie_spm = movie_spm.multiply(topk_movie_spm)
    g = dgl.DGLGraph(topk_movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(topk_movie_spm.data, dtype=torch.float32)
    return g

## Generate graphs for training

In [145]:
conv_g = gen_cosine_graph(user_movie_spm)
loss_g = gen_cooccur_graph(user_movie_spm)
print('co-occur graph has #edges:', loss_g.number_of_edges())

co-occur graph has #edges: 1097853


In [146]:
conv_g.number_of_edges()

22393

Construct the item features.

In [ ]:
year = np.expand_dims(data.movie_data['year'], axis=1)
genre = data.movie_data['genre']
title = data.movie_data['title']
features = torch.tensor(np.concatenate((genre, title), axis=1), dtype=torch.float32)
print('#features:', features.shape[1])
in_feats = features.shape[1]

## GNN models

We run GNN on the item graph to compute item embeddings. In this tutorial, we use a customized [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) model to compute node embeddings. The original GraphSage performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.

The original GraphSage model treats each neighbor equally. However, the SLIM model learns the item similarity based on the user-item iteration. The GNN model should take the similarity into account. Thus, we customize the GraphSage model in the following fashion. Instead of aggregating all neighbors equally, we aggregate neighbors embeddings rescaled by the similarity on the edges. Thus, the aggregation step is defined as follows:

$$h_{N(v)}^{(l)} \gets \Sigma_{u \in N(v)}({h_u^{(l-1)} * s_{uv}}),$$

where $s_{uv}$ is the similarity score between two vertices $u$ and $v$.

The GNN model has multiple layers. In each layer, a vertex accesses its direct neighbors. When we stack $k$ layers in a model, a node $v$ access neighbors within $k$ hops. The output of the GNN model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

We implement the computation in each layer of the customized GraphSage model in `SAGEConv` and implement the multi-layer model in `GraphSAGEModel`.

In [127]:
from sageconv import SAGEConv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()
        if n_layers == 1:
            self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=None))
        elif n_layers > 1:
            # input layer
            self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=activation))
            # hidden layer
            for i in range(n_layers - 2):
                self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type,
                                            feat_drop=dropout, activation=activation))
            # output layer
            self.layers.append(SAGEConv(n_hidden, out_dim, aggregator_type,
                                        feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h, g.edata['similarity'])
        return h

## Train Item Embeddings

We train the item embeddings with the edges in the item graph as the training signal. This step is very similar to the link prediction task in the [basic applications](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

Because the MovieLens dataset has sparse features (both genre and title are stored as multi-hot encoding). The sparse features have many dimensions. To run GNN on the item features, we first create an encoding layer to project the sparse features to a lower dimension. 

In [128]:
class EncodeLayer(nn.Module):
    def __init__(self, in_feats, num_hidden):
        super(EncodeLayer, self).__init__()
        self.proj = nn.Linear(in_feats, num_hidden)
        #self.emb = nn.Embedding(num_movies, num_hidden)
        #self.nid = torch.arange(num_movies).to(device)
        
    def forward(self, feats):
        return self.proj(feats)
        #return torch.cat((self.proj(feats), self.emb(self.nid)), 1)

We simply use node connectivity as the training signal: nodes connected by edges are similar, while nodes not connected by edges are dissimilar.

To train such a model, we need to deploy negative sampling to construct negative samples. A positive sample is an edge that exist in the original graph, while a negative sample is a pair of nodes that don't have an edge between them in the graph. We usually train on each positive sample with multiple negative samples.

After having the node embeddings, we compute the similarity scores on positive samples and negative samples. We construct the following loss function on a positive sample and the corresponding negative samples:

$$L = -log(\sigma(z_u^T z_v)) - Q \cdot E_{v_n \sim P_n(v)}(log(\sigma(-z_u^T z_{v_n}))),$$

where $Q$ is the number of negative samples.

With this loss, training should increase the similarity scores on the positive samples and decrease the similarity scores on negative samples.

In [129]:
# NCE loss
def NCE_loss(pos_score, neg_score, neg_sample_size):
    pos_score = F.logsigmoid(pos_score)
    neg_score = F.logsigmoid(-neg_score).reshape(-1, neg_sample_size)
    return -pos_score - torch.sum(neg_score, dim=1)

class GNNRec(nn.Module):
    def __init__(self, gconv_model):
        super(GNNRec, self).__init__()
        self.encode = EncodeLayer(in_feats, n_hidden)
        self.gconv_model = gconv_model

    def forward(self, conv_g, loss_g, pos_g, neg_g, features, neg_sample_size):
        emb = self.encode(features)
        emb = self.gconv_model(conv_g, emb)
        pos_score = score_func(pos_g, emb)
        neg_score = score_func(neg_g, emb)
        return torch.mean(NCE_loss(pos_score, neg_score, neg_sample_size) * pos_g.edata['weight'])

DGL provides an edge sampler `EdgeSampler`, which selects positive edge samples and negative edge samples efficiently. Thus, we can use it to generate positive samples and negative samples for link prediction. `EdgeSampler` generates `neg_sample_size` negative edges by corrupting the head or the tail node of a positive edge with some randomly sampled nodes.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/negative_edges.png" alt="drawing" width="400"/>

`edge_sampler` samples one tenth of the edges in the graph as positive edges. It returns a positive subgraph and a negative subgraph. The positive subgraph contains all positive edges sampled from the graph `g`, while the negative subgraph contains all negative edges constructed by the edge sampler.

In [130]:
def edge_sampler(g, neg_sample_size, edges=None, return_false_neg=True):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=int(g.number_of_edges()/10),
                                               seed_edges=edges,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               shuffle=True,
                                               return_false_neg=return_false_neg)
    sampler = iter(sampler)
    pos_subg, neg_subg = next(sampler)
    pos_subg.edata['weight'] = g.edata['similarity'][pos_subg.parent_eid]
    return pos_subg, neg_subg

After having the positive edge subgraph and the negative edge subgraph, we can now compute the similarity on the positive edge samples and negative edge samples.

In this tutorial, we use dot-product similarity to measure the similarity between two nodes.

In [131]:
def score_func(g, emb):
    src_nid, dst_nid = g.all_edges(order='eid')
    # Get the node Ids in the parent graph.
    src_nid = g.parent_nid[src_nid]
    dst_nid = g.parent_nid[dst_nid]
    # Read the node embeddings of the source nodes and destination nodes.
    pos_heads = emb[src_nid]
    pos_tails = emb[dst_nid]
    # cosine similarity
    return torch.sum(pos_heads * pos_tails, dim=1)

We evaluate the performance of the trained item embeddings in the item-based recommendation task. We use the last item that a user purchased to represent the user and compute the similarity between the last item and a list of items (an item the user will purchase and a set of randomly sampled items). We calculate the ranking of the item that will be purchased among the list of items.

In [132]:
def RecEvaluate(model, g, features, users_eval, movies_eval, neg_sample_size):
    gconv_model.eval()
    with torch.no_grad():
        emb = model.encode(features)
        emb = model.gconv_model(g, emb)
        hits_10s = []
        # evaluate one user-item interaction at a time
        for u, i in zip(users_eval, movies_eval):
            I_q = user_latest_item[u]
            I = torch.cat([torch.LongTensor([i]), torch.LongTensor(neg_valid[u])])
            Z_q = emb[I_q]
            Z = emb[I]
            score = (Z_q[None, :] * Z).sum(1).cpu().numpy()
            rank = stats.rankdata(-score, 'min')
            hits_10s.append(rank[0] <= 10)
        return np.mean(hits_10s)

Now we put everything in the training loop.

In [144]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

#Model hyperparameters
n_hidden = 8
n_layers = 1
dropout = 0.6
aggregator_type = 'sum'

# create GraphSAGE model
gconv_model = GraphSAGEModel(n_hidden,
                             n_hidden,
                             n_hidden,
                             n_layers,
                             F.leaky_relu,
                             dropout,
                             aggregator_type)
    
# Model for link prediction
model = GNNRec(gconv_model).to(device)
conv_g.to(device)
loss_g.to(device)
features = features.to(device)

# Training hyperparameters
weight_decay = 1e-3
n_epochs = 200
lr = 1e-3
neg_sample_size = 10

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
prev_acc = 0
for epoch in range(n_epochs):
    model.train()
    losses = []
    for pos_subg, neg_subg in dgl.contrib.sampling.EdgeSampler(loss_g, batch_size=1024,
                                               seed_edges=None,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               shuffle=True,
                                               return_false_neg=False):
        pos_subg.edata['weight'] = loss_g.edata['similarity'][pos_subg.parent_eid]
        #pos_subg.edata['weight'] = torch.ones((pos_subg.number_of_edges())).to(device)
        loss = model(conv_g, loss_g, pos_subg, neg_subg, features, neg_sample_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().item())
    
    acc = RecEvaluate(model, conv_g, features, users_valid, movies_valid, neg_sample_size)
    print('Epoch:{}, loss:{:.4f}, HITS@10:{:.4f}'.format(epoch, np.mean(losses), acc))

    # We have an early stop
    #if epoch > 8 and acc <= prev_acc:
    #    break
    #prev_acc = acc

print()
# Let's save the trained node embeddings.
RecEvaluate(model, conv_g, features, users_test, movies_test, neg_sample_size)

Epoch:0, loss:8.4987, HITS@10:0.2967
Epoch:1, loss:8.4535, HITS@10:0.2854
Epoch:2, loss:8.4511, HITS@10:0.2858
Epoch:3, loss:8.4499, HITS@10:0.2969
Epoch:4, loss:8.4493, HITS@10:0.2866
Epoch:5, loss:8.4494, HITS@10:0.3023
Epoch:6, loss:8.4493, HITS@10:0.2974
Epoch:7, loss:8.4496, HITS@10:0.2995
Epoch:8, loss:8.4506, HITS@10:0.3071
Epoch:9, loss:8.4505, HITS@10:0.2930
Epoch:10, loss:8.4501, HITS@10:0.2990
Epoch:11, loss:8.4498, HITS@10:0.3086
Epoch:12, loss:8.4520, HITS@10:0.3111
Epoch:13, loss:8.4497, HITS@10:0.2975
Epoch:14, loss:8.4510, HITS@10:0.2868
Epoch:15, loss:8.4506, HITS@10:0.3003
Epoch:16, loss:8.4512, HITS@10:0.3035


KeyboardInterrupt: 

In [ ]:
emb = model.encode(features)
emb = model.gconv_model(conv_g, emb)
print(emb.cpu().detach().numpy())

In [ ]:
RecEvaluate(model, conv_g, features, users_test, movies_test, neg_sample_size)

In [139]:
from sklearn.metrics.pairwise import cosine_similarity

def knnEvaluate(data, users_eval, movies_eval, neg_sample_size, score_fn):
    hits_10s = []
    # evaluate one user-item interaction at a time
    for u, i in zip(users_eval, movies_eval):
        I_q = user_latest_item[u]
        I = np.concatenate([np.array([i]), neg_valid[u]]).astype(np.int32)
        Z_q = data[I_q]
        if len(Z_q.shape) == 1:
            Z_q = np.expand_dims(Z_q, 1).transpose()
        Z = data[I]
        score = score_fn(Z_q, Z)
        rank = stats.rankdata(-score, 'min')
        hits_10s.append(rank[0] <= 10)
    return np.mean(hits_10s)

In [140]:
hits_10 = knnEvaluate(user_movie_spm.transpose().tocsr(), users_valid, movies_valid, neg_sample_size, cosine_similarity)
print(hits_10)

0.423841059602649


In [ ]:
def dot_score(q, z):
    return (q * z).sum(1)

for d in range(1, 10):
    u, s, vt = spsp.linalg.svds(user_movie_spm, k=d)
    v = vt.transpose() * np.sqrt(s).transpose()
    hits_10 = knnEvaluate(v, users_valid, movies_valid, neg_sample_size, dot_score)
    print('d={}, hits10={}'.format(d, hits_10))

In [ ]:
def dot_score(q, z):
    return (q * z).sum(1)

features_np = pickle.load(open('movielens/movielens_features.pkl', 'rb')).numpy()
data = user_movie_spm.transpose().todense()
data = np.concatenate((data, features_np), 1)

for d in range(1, 10):
    u, s, vt = spsp.linalg.svds(data, k=d)
    u = u * np.sqrt(s).transpose()
    hits_10 = knnEvaluate(u, users_valid, movies_valid, neg_sample_size, dot_score)
    print('d={}, hits10={}'.format(d, hits_10))